In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
import seaborn as sns
from tqdm import tqdm

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import data_quality_checks

In [4]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [5]:
company_df = pd.read_csv('all_retailer_company.csv',index_col = 0)
company_df.columns = ['company_name','company_identifier','state','licenses']
company_df['company_identifier'] = company_df['company_identifier'].astype(str)

In [6]:
company_df_clean = company_df[~company_df['company_identifier'].isin(['PL','WHT','GF','UR'])]
company_df.shape

(66, 4)

In [7]:
len(company_df['company_identifier'].unique())

66

In [8]:
company_df

,company_name,company_identifier,state,licenses
0,99 High Tide,99HT,CA,C10-0000279-LIC
1,Altum LLC,ALT,MI,PC-000310;AU-R-000380
2,Bella Sol,BS,MI,PC-000185;AU-R-000156
3,Big Medicine Cannabissary,BMC,CO,402-00930
4,Boston Bud Factory,BBF,MA,MR281525
...,...,...,...,...
61,Universal Herbal Health Center,UHHC,CA,C10-0000817-LIC;C9-0000082-LIC
62,Uprooted (San Ysidro),UR,CA,C10-0000731-LIC;C10-0000776-LIC;C10-0000246-LI...
63,Verts,VS,CO,402-00840;402R-00545
64,Voyage,VOY,CA,C12-0000030-LIC;C10-0000802-LIC;C12-0000159-LIC


In [9]:
company_df['lic_list'] = [list(company_df['licenses'])[i].split(";") for i in range(len(company_df))]

In [10]:
len(company_df['company_identifier'].unique())

66

# vc

In [11]:
# as of data of 3/2022
VC_START_DATE = '2020-01-01'
VC_END_DATE = '2022-04-01'
VC_MONTH_LIST = ['2021-12-01','2022-01-01','2022-02-01','2022-03-01']
VC_MONTH_END = VC_MONTH_LIST[-1]

In [12]:
def calculate_vendor_churn(incoming_transfer_df,license_list,vc_start_date,vc_end_date,vc_month_list,vc_month_end):
    df_vendor_churn = incoming_transfer_df[incoming_transfer_df['license_number'].isin(license_list)]
    df_vendor_churn['year_month'] = pd.to_datetime(df_vendor_churn['created_date']).dt.strftime("%Y-%m")
    vc = df_vendor_churn[['year_month','shipper_facility_name','shipper_wholesale_price']].groupby(['year_month','shipper_facility_name']).sum().reset_index()
    vc= vc.assign(year_month=lambda df:pd.to_datetime(df['year_month']))

    vc_full = (
        vc
        .groupby('shipper_facility_name').apply(
            lambda df: df.merge(
            pd.Series(
                None, 
                index = pd.date_range(start = vc_start_date, end = vc_end_date, freq = 'MS'), 
                name='__place_holder'
            ), 
            how ='right',
            left_on = 'year_month', 
            right_index=True,
        ).assign(
            **{
                "shipper_facility_name": lambda df_: df_.shipper_facility_name.dropna().unique()[0],
                "shipper_wholesale_price": lambda df_: df_.shipper_wholesale_price.fillna(0),
            }
        )
        .drop('__place_holder', axis=1)
        )
    ).reset_index(drop=True)
    #vc_full[vc_full['shipper_wholesale_price'] < 100]['shipper_wholesale_price'] = 0
    
    indices = vc_full[vc_full['shipper_wholesale_price'] < 100].index.to_list()
    vc_full.loc[indices,'shipper_wholesale_price'] = 0
    
    
    
    rolling_4m_sum =  vc_full.groupby('shipper_facility_name').apply(
    lambda df: df.set_index('year_month').sort_index().rolling(4).sum()
    )
    rolling_4m_sum.columns = ['rolling_4m_total_price']
    facility_monthly_running_total = vc_full.groupby('shipper_facility_name').apply(
        lambda df: df.set_index('year_month').sort_index()['shipper_wholesale_price'].cumsum().to_frame()
    )
    facility_monthly_running_total.columns = ['facility_running_total']
    monthly_running_total = facility_monthly_running_total.reset_index().groupby('year_month')['facility_running_total'].sum().to_frame()
    monthly_running_total.columns = ['monthly_running_total']

    vc_result = rolling_4m_sum.merge(
        facility_monthly_running_total,
        how='inner',
        left_index=True,
        right_index=True
    ).reset_index().merge(
        monthly_running_total,
        how='left',
        on = 'year_month'
    )
    vc_result['%_total'] = vc_result['facility_running_total'] / vc_result['monthly_running_total']
    vc_result['last_4m_active'] = vc_result['rolling_4m_total_price'] > 0
    vc_result ['significant'] = vc_result['%_total'] > 0.001
    vc_result['measure'] = vc_result.apply(
    lambda row: "Active"  if (row['last_4m_active'] & row['significant']) else ("Inactive" if row['significant'] else "Exclude"),
    axis=1
    )
    #churn
    churn = vc_result.groupby(['year_month']).apply(lambda x: x[x['measure'] == 'Inactive']['%_total'].sum()).reset_index()
    churn.columns = ['year_month','%_inactive']
    churn.index = churn.year_month
    
    # output vendor churn matrix
    vc_data = vc_full[vc_full['year_month'] <= VC_MONTH_END]
    vc_data['year_month'] = vc_data['year_month'].astype(str)
    vc_matrix = pd.pivot_table(vc_data, values='shipper_wholesale_price', index='shipper_facility_name',
    columns='year_month', fill_value=0).reset_index()
    vc_matrix['facility_total'] = vc_matrix.sum(axis= 1)
    vc_matrix['grand_total'] = vc_matrix['facility_total'].sum()
    vc_matrix['perc_total'] = vc_matrix['facility_total'] / vc_matrix['grand_total']
    vc_matrix['last_4m_total'] = vc_matrix[VC_MONTH_LIST].sum(axis = 1)
    vc_matrix['last_4m_active'] = vc_matrix['last_4m_total'] > 0
    vc_matrix ['significant'] = vc_matrix['perc_total'] > 0.001
    vc_matrix['measure'] = vc_matrix.apply(
        lambda row: "Active"  if (row['last_4m_active'] & row['significant']) else ("Inactive" if row['significant'] else "Exclude"),
        axis=1
    )
    return churn, vc_matrix


In [13]:
def calculate_vendor_churn_short(incoming_transfer_df,license_list,vc_start_date,vc_end_date,vc_month_list,vc_month_end):
    df_vendor_churn = incoming_transfer_df[incoming_transfer_df['license_number'].isin(license_list)]
    df_vendor_churn['year_month'] = pd.to_datetime(df_vendor_churn['created_date']).dt.strftime("%Y-%m")
    vc = df_vendor_churn[['year_month','shipper_facility_name','shipper_wholesale_price']].groupby(['year_month','shipper_facility_name']).sum().reset_index()
    vc= vc.assign(year_month=lambda df:pd.to_datetime(df['year_month']))

    vc_full = (
        vc
        .groupby('shipper_facility_name').apply(
            lambda df: df.merge(
            pd.Series(
                None, 
                index = pd.date_range(start = vc_start_date, end = vc_end_date, freq = 'MS'), 
                name='__place_holder'
            ), 
            how ='right',
            left_on = 'year_month', 
            right_index=True,
        ).assign(
            **{
                "shipper_facility_name": lambda df_: df_.shipper_facility_name.dropna().unique()[0],
                "shipper_wholesale_price": lambda df_: df_.shipper_wholesale_price.fillna(0),
            }
        )
        .drop('__place_holder', axis=1)
        )
    ).reset_index(drop=True)
    #vc_full[vc_full['shipper_wholesale_price'] < 100]['shipper_wholesale_price'] = 0
    
    
    indices = vc_full[vc_full['shipper_wholesale_price'] < 100].index.to_list()
    vc_full.loc[indices,'shipper_wholesale_price'] = 0
    
    #rolling_4m_sum
    rolling_4m_sum =  vc_full.groupby('shipper_facility_name').apply(
    lambda df: df.set_index('year_month').sort_index().rolling(4).sum()
    )
    rolling_4m_sum.columns = ['rolling_4m_total_price']
    
    #facility_monthly_running_total
    facility_monthly_running_total = vc_full.groupby('shipper_facility_name').apply(
        lambda df: df.set_index('year_month').sort_index()['shipper_wholesale_price'].rolling(window = 12).sum().to_frame()
    )
    facility_monthly_running_total.columns = ['facility_running_total']
    
    #monthly_running_total
    monthly_running_total = facility_monthly_running_total.reset_index().groupby('year_month')['facility_running_total'].sum().to_frame()
    monthly_running_total.columns = ['monthly_running_total']

    vc_result = rolling_4m_sum.merge(
        facility_monthly_running_total,
        how='inner',
        left_index=True,
        right_index=True
    ).reset_index().merge(
        monthly_running_total,
        how='left',
        on = 'year_month'
    )
    vc_result['%_total'] = vc_result['facility_running_total'] / vc_result['monthly_running_total']
    vc_result['last_4m_active'] = vc_result['rolling_4m_total_price'] > 0
    vc_result ['significant'] = vc_result['%_total'] > 0.001
    vc_result['measure'] = vc_result.apply(
    lambda row: "Active"  if (row['last_4m_active'] & row['significant']) else ("Inactive" if row['significant'] else "Exclude"),
    axis=1
    )
    #churn
    churn = vc_result.groupby(['year_month']).apply(lambda x: x[x['measure'] == 'Inactive']['%_total'].sum()).reset_index()
    churn.columns = ['year_month','%_inactive']
    churn.index = churn.year_month
    
    # output vendor churn matrix
    vc_data = vc_full[vc_full['year_month'] <= VC_MONTH_END]
    vc_data['year_month'] = vc_data['year_month'].astype(str)
    vc_matrix = pd.pivot_table(vc_data, values='shipper_wholesale_price', index='shipper_facility_name',
    columns='year_month', fill_value=0).reset_index()
    vc_matrix['facility_total'] = vc_matrix.iloc[: , -12:].sum(axis = 1)
    vc_matrix['grand_total'] = vc_matrix['facility_total'].sum()
    vc_matrix['perc_total'] = vc_matrix['facility_total'] / vc_matrix['grand_total']
    vc_matrix['last_4m_total'] = vc_matrix[vc_month_list].sum(axis = 1)
    vc_matrix['last_4m_active'] = vc_matrix['last_4m_total'] > 0
    vc_matrix ['significant'] = vc_matrix['perc_total'] > 0.001
    vc_matrix['measure'] = vc_matrix.apply(
        lambda row: "Active"  if (row['last_4m_active'] & row['significant']) else ("Inactive" if row['significant'] else "Exclude"),
        axis=1
    )
    return churn, vc_matrix
    


In [ ]:
## test

In [ ]:
#bad example of small number mess up whole vc%
# to do: impose minimal amount to the vc matrix
df_in_test,df_sales_deduped_test,bad_dl_test,unknown_transfer_test,rwp_test,miss_incoming_test,miss_receipts_test = data_quality_checks.run(['KC'],TRANSFER_PACKAGES_START_DATE,SALES_TRANSACTIONS_START_DATE)




In [ ]:
lic_test = df_in_test['license_number'].unique()[8]
lic_test


In [ ]:
a_short,b_short,f_short = calculate_vendor_churn_short(df_in_test,[lic_test],VC_START_DATE,VC_END_DATE,VC_MONTH_LIST,VC_MONTH_END)

a,b,f = calculate_vendor_churn(df_in_test,[lic_test],VC_START_DATE,VC_END_DATE,VC_MONTH_LIST,VC_MONTH_END)





In [ ]:
#c,vc,a,b = calculate_vendor_churn(df_in,[lic],VC_START_DATE,VC_END_DATE,VC_MONTH_LIST,VC_MONTH_END)




# run

In [14]:
def read_df_in(COMPANY_IDENTIFIER,license_numbers):
    company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
    )
    company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
    return company_incoming_transfer_packages_dataframe


In [15]:
def get_vc_list(company_list,transfer_packages_start_date,sales_transactrions_start_date,map_df):
    churn_df_list = {}
    vc_matrix_df_list = {}
    churn_df_list_short = {}
    vc_matrix_df_list_short = {}
    for c in tqdm(company_list):
        churn_list = {}
        vc_matrix_list = {}
        churn_list_short = {}
        vc_matrix_list_short = {}
        print(c)
        l_list = map_df[map_df['company_identifier'] == c]['lic_list'].to_list()[0]
        for l in tqdm(l_list):
            print(l)
            df_in = read_df_in([c],[l])
            if df_in.shape[0] == 0:
                churn_list[l] = numpy.nan
                vc_matrix_list[l] = numpy.nan
                churn_list_short[l] = numpy.nan
                vc_matrix_list_short[l] = numpy.nan
                continue
            else:
                churn,vc_matrix = calculate_vendor_churn(df_in,[l],VC_START_DATE,VC_END_DATE,VC_MONTH_LIST,VC_MONTH_END)
                churn_short,vc_matrix_short = calculate_vendor_churn_short(df_in,[l],VC_START_DATE,VC_END_DATE,VC_MONTH_LIST,VC_MONTH_END)
                churn_list[l] = churn
                vc_matrix_list[l] = vc_matrix
                churn_list_short[l] = churn_short
                vc_matrix_list_short[l] = vc_matrix_short
        churn_df_list[c] = churn_list
        vc_matrix_df_list[c] = vc_matrix_list
        churn_df_list_short[c] = churn_list_short
        vc_matrix_df_list_short[c] = vc_matrix_list_short
    return churn_df_list,vc_matrix_df_list,churn_df_list_short,vc_matrix_df_list_short

    

    

In [16]:
c,vc,c_short,vc_short = get_vc_list(list(company_df['company_identifier']),TRANSFER_PACKAGES_START_DATE,SALES_TRANSACTIONS_START_DATE,company_df)




  0%|          | 0/66 [00:00<?, ?it/s]

99HT



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000279-LIC



  2%|▏         | 1/66 [00:04<04:51,  4.48s/it]

...


 98%|█████████▊| 65/66 [09:17<00:14, 14.09s/it]

WHT



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000548-LIC



100%|██████████| 66/66 [09:23<00:00,  8.53s/it]


In [ ]:
df_vc_churn = pd.DataFrame()
for k in vc.keys():
    print(k)
    for l in vc[k].keys():
        print(l)
        data_vc = vc[k][l]
        if str(data_vc) == 'nan':
            continue
        else:
            data_vc['company'] = k
            data_vc['location'] = l
            print(data_vc.shape)
            df_vc_churn = df_vc_churn.append(data_vc)
        #data.to_excel('ca_location_analysis/'+'analysis_' + k + '_'+ l+'.xlsx')

In [ ]:
df_churn = pd.DataFrame()
for k in c.keys():
    print(k)
    for l in c[k].keys():
        print(l)
        data_vc = c[k][l]
        if str(data_vc) == 'nan':
            continue
        else:
            data_vc['company'] = k
            data_vc['location'] = l
            print(data_vc.shape)
            df_churn = df_churn.append(data_vc)
        #data.to_excel('ca_location_analysis/'+'analysis_' + k + '_'+ l+'.xlsx')

In [ ]:
df_churn_short = pd.DataFrame()
for k in c_short.keys():
    print(k)
    for l in c_short[k].keys():
        print(l)
        data_vc_short = c_short[k][l]
        if str(data_vc_short) == 'nan':
            continue
        else:
            data_vc_short['company'] = k
            data_vc_short['location'] = l
            print(data_vc_short.shape)
            df_churn_short = df_churn_short.append(data_vc_short)
        #data.to_excel('ca_location_analysis/'+'analysis_' + k + '_'+ l+'.xlsx')

In [ ]:
df_churn_short[df_churn_short['company'] == 'EL']

# analysis: past 6m

In [ ]:
df_churn_short_recent = df_churn_short[(df_churn_short['year_month'] >= '2021-09') & (df_churn_short['year_month'] <= '2022-02')]
df_churn_short_recent = df_churn_short_recent.reset_index(drop = True)

df_churn_recent = df_churn[(df_churn['year_month'] >= '2021-09') & (df_churn['year_month'] <= '2022-02')]
df_churn_recent = df_churn_recent.reset_index(drop = True)




In [ ]:
df_churn_short_recent_with_state = pd.merge(df_churn_short_recent,company_df[['company_identifier','state']],left_on = 'company',right_on = 'company_identifier',how = 'left')
df_churn_recent_with_state = pd.merge(df_churn_recent,company_df[['company_identifier','state']],left_on = 'company',right_on = 'company_identifier',how = 'left')




In [ ]:
df_churn_short_recent_with_state.head()


In [ ]:
df_churn_recent_with_state.head()

In [ ]:
churn_joined = pd.merge(df_churn_recent_with_state,df_churn_short_recent_with_state, on = ['year_month','company','location','company_identifier','state'],how = 'inner')

churn_joined.columns = ['year_month','churn_orig','company','location','company_identifier','state','churn_new']

churn_joined ['new-old'] = churn_joined['churn_new'] - churn_joined['churn_orig']




In [ ]:
churn_joined.sort_values(by = 'new-old')

In [ ]:
# distribution of margin short
churn_distr_short = df_churn_short_recent_with_state.groupby(['state','year_month'])['%_inactive'].describe().reset_index()
churn_distr_short['year_month'] = pd.to_datetime(churn_distr_short['year_month'])
churn_distr_short.index = churn_distr_short.year_month


# distribution of margin orig
churn_distr = df_churn_recent_with_state.groupby(['state','year_month'])['%_inactive'].describe().reset_index()
churn_distr['year_month'] = pd.to_datetime(churn_distr['year_month'])
churn_distr.index = churn_distr.year_month



In [ ]:
churn_distr[churn_distr['state'] == 'CO']

In [ ]:
churn_distr_short[churn_distr_short['state'] == 'CO']

In [ ]:
import seaborn as sns

In [ ]:
churn_distr[churn_distr['state'] == 'CA']['50%'].plot(marker = 'o',label = 'churn original')
churn_distr_short[churn_distr_short['state'] == 'CA']['50%'].plot(marker = 'o',label = 'churn new')
plt.legend()
ax = plt.ylim(0.05,0.6)

In [ ]:
churn_distr[churn_distr['state'] == 'CA']['mean'].plot(marker = 'o',label = 'churn original')
churn_distr_short[churn_distr_short['state'] == 'CA']['mean'].plot(marker = 'o',label = 'churn new')
plt.legend()
ax = plt.ylim(0.05,0.6)

In [ ]:
churn_distr[churn_distr['state'] == 'CO']['50%'].plot(marker = 'o',label = 'churn original')
churn_distr_short[churn_distr_short['state'] == 'CO']['50%'].plot(marker = 'o',label = 'churn new')
plt.legend()
ax = plt.ylim(0.05,0.6)

In [ ]:
churn_distr[churn_distr['state'] == 'CO']['mean'].plot(marker = 'o',label = 'churn original')
churn_distr_short[churn_distr_short['state'] == 'CO']['mean'].plot(marker = 'o',label = 'churn new')
plt.legend()
ax = plt.ylim(0.05,0.6)

# Qualification rate

In [ ]:
df_clients = [
'DL',
'DW',
'EMA',
'EMM',
'EMT',
'EMF',
'ST',
'GRG',
'EL',
'VS',
'TT',
'MD',
'DWF',
'GHC',
'SV',
'99HT'
]

In [ ]:
churn_joined_client = churn_joined[churn_joined['company_identifier'].isin(df_clients)]

In [ ]:
churn_joined_client

In [ ]:
# distribution of margin short for current df clients
churn_distr_short_df = churn_joined_client.groupby(['state','year_month'])['churn_new'].describe().reset_index()
churn_distr_short_df['year_month'] = pd.to_datetime(churn_distr_short_df['year_month'])
churn_distr_short_df.index = churn_distr_short_df.year_month


# distribution of margin orig for current df clients
churn_distr_df = churn_joined_client.groupby(['state','year_month'])['churn_orig'].describe().reset_index()
churn_distr_df['year_month'] = pd.to_datetime(churn_distr_df['year_month'])
churn_distr_df.index = churn_distr_df.year_month



In [ ]:
churn_distr_df[churn_distr_df['state'] == 'CA']['50%'].plot(marker = 'o',label = 'churn original')
churn_distr_short_df[churn_distr_short_df['state'] == 'CA']['50%'].plot(marker = 'o',label = 'churn new')
plt.legend()
ax = plt.ylim(0.05,0.3)

In [ ]:
churn_distr_df[churn_distr_df['state'] == 'CA']['mean'].plot(marker = 'o',label = 'churn original')
churn_distr_short_df[churn_distr_short_df['state'] == 'CA']['mean'].plot(marker = 'o',label = 'churn new')
plt.legend()
ax = plt.ylim(0.05,0.3)

In [ ]:
churn_distr_df[churn_distr_df['state'] == 'CA']

In [ ]:
churn_distr_short_df[churn_distr_short_df['state'] == 'CA']

In [ ]:
churn_distr_df[churn_distr_df['state'] == 'CO']

In [ ]:
churn_distr_short_df[churn_distr_short_df['state'] == 'CO']

In [ ]:
churn_distr_df[churn_distr_df['state'] == 'CO']['50%'].plot(marker = 'o',label = 'churn original')
churn_distr_short_df[churn_distr_short_df['state'] == 'CO']['50%'].plot(marker = 'o',label = 'churn new')
plt.legend()
ax = plt.ylim(0,0.3)

In [ ]:
churn_distr_df[churn_distr_df['state'] == 'CO']['mean'].plot(marker = 'o',label = 'churn original')
churn_distr_short_df[churn_distr_short_df['state'] == 'CO']['mean'].plot(marker = 'o',label = 'churn new')
plt.legend()
ax = plt.ylim(0,0.3)

In [ ]:
def calculate_monthly_qualification(data):
    q_df = {}
    nq_df = {}
    new_q_df = {}
    new_nq_df = {}
    
    q_2_nq_df = {}
    nq_2_q_df = {}
    for m in ['2021-09-01','2021-10-01','2021-11-01',\
             '2021-12-01','2022-01-01','2022-02-01']:
        
        d = data[data['year_month'] == m]
        qualify_list = d[d['churn_orig'] < 0.45]['company'].unique()
        non_qualify_list = d[d['churn_orig'] >= 0.45]['company'].unique()
        new_qualify_list = d[d['churn_new'] < 0.45]['company'].unique()
        new_non_qualify_list = d[d['churn_new'] >= 0.45]['company'].unique()
        
        q_df[m] = qualify_list
        nq_df[m] = non_qualify_list
        new_q_df[m] = new_qualify_list
        new_nq_df[m] = new_non_qualify_list
        
        q_2_nq = set(qualify_list) - set(new_qualify_list)
        nq_2_q = set(non_qualify_list) - set(new_non_qualify_list)
        
        q_2_nq_df[m] = q_2_nq
        nq_2_q_df[m] = nq_2_q
    return q_df,nq_df,new_q_df,new_nq_df,q_2_nq_df,nq_2_q

In [ ]:
q,nq,new_q,new_nq,q_2_nq_data,nq_2_q_data = calculate_monthly_qualification(churn_joined_client)

In [ ]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in q.items() ])).T

In [ ]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in nq.items() ])).T

In [ ]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in new_q.items() ])).T

In [ ]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in new_nq.items() ])).T